In [ ]:
import sys
import os


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
print(sys.path[-1])

In [ ]:
from src.classifiers import pipeline
import pandas as pd

In [ ]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(BASE_DIR, "data")
MODELS_DIR = os.path.join(BASE_DIR, "models")
RESULTS_DIR = os.path.join(BASE_DIR, "results")
CLASSIFIERS = [
    "LogisticRegression",
    "GaussianNB",
    "LinearDiscriminantAnalysis",
    "SVC",
    "RandomForestClassifier",
    "LGBMClassifier"
]
DF = pd.read_csv(os.path.join(DATA_DIR, "breast_cancer.csv"))
TARGET = "diagnosis"
N_ROUNDS = 10
N_OUTER_FOLDS = 5
N_INNER_FOLDS = 3
N_FEATURES_TO_SELECT = 5
N_OPTUNA_TRIALS = 50
VALIDATION_SET_FRACTION = 0.1
SEED = 42

In [ ]:
val_set = pipeline(
        classifiers=CLASSIFIERS,
        df=DF,
        target=TARGET,
        models_dir=MODELS_DIR,
        results_dir=RESULTS_DIR,
        n_rounds=N_ROUNDS,
        n_outer_folds=N_OUTER_FOLDS,
        n_inner_folds=N_INNER_FOLDS,
        n_features_to_select=N_FEATURES_TO_SELECT,
        n_optuna_trials=N_OPTUNA_TRIALS,
        validation_set_fraction=VALIDATION_SET_FRACTION,
        seed=SEED
)